The goal of this notebook is to check the performance of various formulations

In [ ]:
using Pkg; Pkg.activate()
using Serialization
using nGraph, Runner, Zoo
using JuMP, Interact, DataStructures

using Plots
gr()

In [ ]:
function perf_plot(simple, synchronous; title = "")
    io_size = simple.io_sizes[]

    # Sanity check
    @assert io_size == synchronous.io_sizes[]

    # Show the IO size in GB
    @show io_size / 1E9

    total_dram = simple.default_alloc_size[] + io_size

    dram_performance = min(
        minimum(simple.actual_runtimes),
        minimum(synchronous.actual_runtimes),
    )

    plt = plot(
        (simple.dram_alloc_size .+ io_size) ./ total_dram,
        (simple.actual_runtimes ./ dram_performance),
        marker = :square,
        label = "Static ILP",
        minorgrid = true,
    )

    plot!(plt,
        (synchronous.dram_alloc_size .+ io_size) ./ total_dram,
        (synchronous.actual_runtimes ./ dram_performance),
        marker = :o,
        label = "Synchronous ILP",
    )

    title!(plt, title)
    ylabel!(plt, "Relative Performance")
    xlabel!(plt, "Fraction of Total DRAM")

    return plt
end

In [ ]:
simple_data = deserialize("serials/simple_resnet50_256.jls")
synchronous_data = deserialize("serials/synchronous_resnet50_256.jls")
title = "Resnet 50 - Batchsize 256"

perf_plot(simple_data, synchronous_data; title = title)

In [ ]:
simple_data = deserialize("serials/simple_resnet200_256.jls")
#synchronous_data = deserialize("serials/synchronous_resnet50_256.jls")
title = "Resnet 200 - Batchsize 256"

perf_plot(simple_data, synchronous_data; title = title)

# Kernel Timings

In [ ]:
#f = () -> Zoo.inception_v4_training(3072)
#f = () -> Zoo.resnet_training(50, 256)
#f = () -> Zoo.vgg19_training(128)
f = () -> Zoo.inception_v4_training(128)

# Generator functions for the various optimization methods
simple(n) = function(data)
    bounds = Runner.allocation_bounds(data)
    x = round(Int, bounds.upper_bound * n / 1E6)
    return Runner.Simple(x)
end

synchronous(n) = function(data)
    bounds = Runner.allocation_bounds(data)
    x = round(Int, bounds.upper_bound * n / 1E6)
    println("Trying to use $x MB of memory")
    return Runner.Synchronous(x, 29000, 12000)
end

In [ ]:
#ratios = [0.0, 0.02, 0.05, 0.1, 0.2, 1.0]
ratios = [0.05]
exe_frame_pairs = map(ratios) do r
    fex, args, frame, metadata = Runner.factory(f, simple(r), Runner.OnlyIntermediate())
    return (fex = fex, frame = frame)
end

In [ ]:
# Do a marginal cost plot
pair = exe_frame_pairs[1]
fex = pair.fex
data = pair.frame.profile_data

@time marginals = Runner.Analyzer.marginal_costs(fex, data)
plot(marginals)

# Kernel Plots

In [ ]:
function genplot(kernels::Vector{Runner.Analyzer.Kernel}, filt)

    # Filter along each inner vector
    filtered_kernels = filter(
        x -> isequal(Runner.filter_out_io(x.params), Runner.filter_out_io(filt.params)), kernels)

    nrows = (kernels) -> 1
    ncols = (kernels) -> length(first(kernels).config)
    x = (kernels, row, col) -> [
        vcat(
            collect(prod.(k.params.input_sizes)),
            collect(prod.(k.params.output_sizes))
        )[col] for k in kernels
    ]
    y = (kernels, row, col) -> [k.index for k in kernels]
    color = (kernels, row, col) -> [k.config[col] == Runner.DRAM ? :blue : :red for k in kernels]
    title = function(kernels, row, col)
        k = first(kernels)
        if col <= length(k.params.input_formats)
            format = last(split(nGraph.Lib.get_mkldnn_string(k.params.input_formats[col]), ":"))
        else
            format = "is output"
        end
        return "$format"
    end

    KP = Runner.Analyzer.KernelPlot(filtered_kernels, nrows, ncols, x, y, color, title)
    plot(KP)
end

In [ ]:
executables = getproperty.(exe_frame_pairs, :fex)
kernels = Runner.Analyzer.kernel_breakdown.(executables)
unique_kernels = unique(x -> Runner.filter_out_io(x.params), first(kernels))

# Make options to select the kernel
options = OrderedDict("$i $(Runner.Analyzer.description(k))" => i for (i,k) in enumerate(unique_kernels))

mp = @manipulate for v in dropdown(options), ratio in ratios
    r = findfirst(isequal(ratio), ratios)
    genplot(kernels[r], unique_kernels[v])
end

## Reuse Distance

In [ ]:
md = @manipulate for ratio in ratios
    r = findfirst(isequal(ratio), ratios)
    
    # Unpack FluxExecutable and ProfileData
    pair = exe_frame_pairs[r]
    fex = pair.fex
    data = pair.frame.profile_data
    
    rd = Runner.Analyzer.reuse_distance(fex, data)
    plot(rd)
end